In [1]:
import sys, os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')

print(train_df.head())


                                               image                   id  \
0  ../../data/Train/CameraRGB/episode_0002_000287...  episode_0002_000287   
1  ../../data/Train/CameraRGB/episode_0008_000112...  episode_0008_000112   
2                 ../../data/Train/CameraRGB/804.png                  804   
3  ../../data/Train/CameraRGB/episode_0008_000286...  episode_0008_000286   
4               ../../data/Train/CameraRGB/M2-25.png                M2-25   

                                               label  
0  ../../data/Train/CameraSeg/episode_0002_000287...  
1  ../../data/Train/CameraSeg/episode_0008_000112...  
2                 ../../data/Train/CameraSeg/804.png  
3  ../../data/Train/CameraSeg/episode_0008_000286...  
4               ../../data/Train/CameraSeg/M2-25.png  


In [4]:
from models.fcn8 import model_fcn8

model = model_fcn8(3, image_shape=(416, 800, 3), keep_prob=0.5)
model.summary()

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 800, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 416, 800, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 416, 800, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 208, 400, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [5]:
from gen.datagen import oversample_generator_from_df, balanced_generator_from_df

BATCH_SIZE = 16
model_dir = '../../saved_models/fcn8/fcn8_v10/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

train_gen = oversample_generator_from_df(train_df, BATCH_SIZE, (416, 800))
valid_gen = balanced_generator_from_df(valid_df, BATCH_SIZE, (416, 800))

In [6]:
from train import weighted_categorical_crossentropy
from keras.utils.training_utils import multi_gpu_model

gpus = 1
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [7]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model, 
                   train_gen, 
                   valid_gen, 
                   training_size=4000, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                   gpus = gpus)

Instructions for updating:
Use the retry module or similar alternatives.
Epoch 1/100
250/250 [==============================] - 369s 1s/step - loss: 1.6320 - acc: 0.8582 - val_loss: 0.6102 - val_acc: 0.9097

Epoch 00001: val_loss improved from inf to 0.61024, saving model to ../../saved_models/fcn8/fcn8_v10//model.hdf5
Epoch 2/100
250/250 [==============================] - 298s 1s/step - loss: 0.3529 - acc: 0.9377 - val_loss: 0.3437 - val_acc: 0.9217

Epoch 00002: val_loss improved from 0.61024 to 0.34369, saving model to ../../saved_models/fcn8/fcn8_v10//model.hdf5
Epoch 3/100
250/250 [==============================] - 299s 1s/step - loss: 0.2052 - acc: 0.9449 - val_loss: 0.3484 - val_acc: 0.9182

Epoch 00003: val_loss did not improve
Epoch 4/100
250/250 [==============================] - 284s 1s/step - loss: 0.1764 - acc: 0.9492 - val_loss: 0.2876 - val_acc: 0.9268

Epoch 00004: val_loss improved from 0.34369 to 0.28761, saving model to ../../saved_models/fcn8/fcn8_v10//model.hdf5
Ep

KeyboardInterrupt: 

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [8]:
model.load_weights('../../saved_models/fcn8/fcn8_v10//model.hdf5')
model.save('../../saved_models/fcn8/fcn8_v10/model_saved.h5')

In [ ]:
#valid_gen = train_and_lab_gen_func(valid_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE * gpus)
#test_gen = train_and_lab_gen_func(test_df, image_size=(600, 800),  target_size=(480, 480),  batch_size = BATCH_SIZE * gpus)